<a href="https://colab.research.google.com/github/thunguyen177/DPER/blob/main/DPER_with_equal_covariance_assumption_FASHION_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## libraries and function 




In [ ]:
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
import impyute as impy
from fancyimpute import SoftImpute, NuclearNormMinimization
import pandas as pd
import time

## MLE estimation function  




In [ ]:
def diag_term(i,X,y):
  arr0 = X[:,i]
  nar2 = 0
  arr = arr0[~np.isnan(arr0)]
  y_arr = y[~np.isnan(arr0)]

  _, counts = np.unique(y_arr, return_counts=True)
  ind = np.insert(np.cumsum(counts), 0, 0)
  
  return sum([(ind[g]-ind[g-1])*np.var(arr[ind[g-1]:ind[g]]) for 
                       g in range(1,G+1)])/len(y_arr)                       

In [ ]:
def mle(X,y,G):
    '''
    X: input, should be a numpy array
    y: label
    G: number of classes
    output:
    - mus: each row is a class mean
    - S: common covariance matrix of class 1,2,..., G 
    '''
    epsilon = 1e-5 # define epsilon to put r down to 0 if r < epsilon
    n,p = X.shape[0], X.shape[1]

    mus = np.array([np.nanmean(X[y==g,:],axis=0) for g in range (G)]).T # so that each column is the mean of a class
 

    S = np.diag([diag_term(i,X,y) for i in range(p)]) 

    for i in range(p):      
      for j in range(i):
        mat = X[:,[i,j]]

        # drop rows with NA
        idx = ~np.isnan(mat).any(axis=1)
        mat, y_arr = mat[idx], y[idx]

        _, counts = np.unique(y_arr, return_counts=True)
        ind = np.insert(np.cumsum(counts), 0, 0)

        m_g = counts
        
        A = len(y_arr)

        scaled_mat = [mat[ind[g-1]:ind[g],:]-mus[[i,j],g-1] for g in range(1,G+1)]
    
        q = lambda g: np.dot(scaled_mat[g][:,0],scaled_mat[g][:,0])
        s11 = sum(map(q,range(G))) 
        q = lambda g: np.dot(scaled_mat[g][:,1],scaled_mat[g][:,1])
        s22 = sum(map(q,range(G))) 
        d = lambda g: np.dot(scaled_mat[g][:,0],scaled_mat[g][:,1])
        s12 = sum(map(d,range(G)))  

        start_solve = time.time()
        B = S[i,i]*S[j,j]*A - s22 * S[i,i] - s11 * S[j,j]
        coefficient = [-A, s12, B, s12*S[i,i]*S[j,j]]
        r = np.roots(coefficient)

        r = r[abs(np.imag(r)) < epsilon]
        r = np.real(r)
        r[abs(r) < epsilon] = 0

        if len(r)>1:
          condi_var = S[j,j] - r**2/S[i,i]
          eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 +
                                      r**2/S[i,i]**2*s11)/condi_var
          # if condi_var <0 then eta = NA. in practice, it's impossible for cov to be negative
          #  therefore, we drop NA elements of eta  
          r = r[eta == max(eta[~np.isnan(eta)])]

        if len(r) > 1:        
            w = [m_g[g-1]*np.cov(mat[ind[g-1]:ind[g],], rowvar=False) for
                 g in range(1,G+1)]
            w = np.sum(w, axis = 0)   
            r = r[np.abs(r-w[0,1]).argmin()] # choose r that is w[0,1] 
              
        S[i,j] = S[j,i] = r
    return [mus, S]

### compute_err function 
  

In [ ]:
def err(mus, S, mus_est, S_est):
  er = [np.linalg.norm(mus_est-mus)/mus.size,
         np.linalg.norm(S_est.flatten()-S.flatten())/S.size]
  return np.mean(er)  

def generate_nan(Xtrain, missing_rate):
  Xshape = Xtrain.shape
  na_id = np.random.randint(0,Xtrain.size,round(missing_rate*Xtrain.size))
  Xtr_nan = Xtrain.flatten()
  Xtr_nan[na_id] = np.nan 
  return Xtr_nan.reshape(Xshape)

### Algorithm to compute error for DPER only

In [ ]:
def compute_err_mforest(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [sum(ytrain==g)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.sum(S, axis = 0)/len(ytrain)

    start = time.time()
    Xtr_mforest = MissForest(random_state=0).fit_transform(Xtr_nan)
    mus_mforest = np.asarray([np.mean(Xtr_mforest[ytrain==g,:], axis=0
                                         ) for g in np.arange(G)])
    S_mforest = np.asarray([(sum(ytrain==g))*np.cov(Xtr_mforest[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_mforest = np.sum(S_mforest, axis = 0)/len(ytrain)   
    mforest_err =  err(mus, S, mus_mforest, S_mforest)
    mforest_time = time.time()-start   
    return mforest_err, mforest_time

In [ ]:
def compute_err_dper(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    #Sort dataset by y_train
    arsort = ytrain.argsort()
    ytrain = ytrain[arsort]
    Xtrain = Xtrain[arsort]
    Xtr_nan = Xtr_nan[arsort]
    
    #Find indices
    _, counts = np.unique(ytrain, return_counts=True)
    ind = np.insert(np.cumsum(counts), 0, 0)

    mus = np.array([np.mean(Xtrain[ind[g-1]:ind[g]],axis=0) for g in range(1,G+1)])
    S = [(ind[g]-ind[g-1])*np.cov(Xtrain[ind[g-1]:ind[g],:],rowvar = False) for g in range(1,G+1)]
    S = np.sum(S, axis = 0)/len(ytrain)

    # MLEs approach
    start = time.time()
    mus_mle, S_mle = mle(Xtr_nan,ytrain, G)
    mle_err = err(mus, S, mus_mle.T, S_mle)
    mle_time = time.time()-start     
    return mle_err, mle_time

In [ ]:
def compute_err_soft(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [sum(ytrain==g)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.sum(S, axis = 0)/len(ytrain)

    start = time.time()
    Xtr_softimpute = SoftImpute(max_iters = 100).fit_transform(Xtr_nan)
    mus_softimpute = np.asarray([np.mean(Xtr_softimpute[ytrain==g,:], axis=0
                                         ) for g in np.arange(G)])
    S_softimpute = np.asarray([(sum(ytrain==g))*np.cov(Xtr_softimpute[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_softimpute = np.sum(S_softimpute, axis = 0)/len(ytrain)   
    softimpute_err =  err(mus, S, mus_softimpute, S_softimpute)
    softimpute_time = time.time()-start   
    return softimpute_err, softimpute_time

In [ ]:
def compute_err_mice(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [sum(ytrain==g)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.sum(S, axis = 0)/len(ytrain)

    start = time.time()
    Xtr_mice = IterativeImputer(max_iter=10).fit(Xtr_nan).transform(Xtr_nan)
    mus_mice = np.asarray([np.mean(Xtr_mice[ytrain==g,:], axis=0
                                   ) for g in np.arange(G)])
    S_mice = np.asarray([(sum(ytrain==g))*np.cov(Xtr_mice[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_mice = np.sum(S_mice, axis = 0)/len(ytrain)
    mice_err = err(mus, S, mus_mice, S_mice)
    mice_time = time.time()-start

    return mice_err, mice_time

In [ ]:
def compute_err_nuclear(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [sum(ytrain==g)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.sum(S, axis = 0)/len(ytrain)

    start = time.time()
    Xtr_nuclear = NuclearNormMinimization(max_iters=10).fit_transform(Xtr_nan)
    mus_nuclear = np.asarray([np.mean(Xtr_nuclear[ytrain==g,:], axis=0
                                      ) for g in np.arange(G)])
    S_nuclear = np.asarray([(sum(ytrain==g))*np.cov(Xtr_nuclear[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_nuclear = np.sum(S_nuclear, axis = 0)/len(ytrain)
    nuclear_err = err(mus, S, mus_nuclear, S_nuclear)
    nuclear_time = time.time()-start

    return nuclear_err, nuclear_time

# FASHION MNIST

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

fashion_mnist = tf.keras.datasets.fashion_mnist
(Xtrain, ytrain), (Xtest, ytest) = fashion_mnist.load_data()

In [ ]:
Xtrain = Xtrain.astype(float).reshape((60000,784))
Xtest = Xtest.astype(float).reshape((10000,784))

X = np.vstack((Xtrain, Xtest))
y = np.hstack((ytrain, ytest))

# set random seed and shuffle the data
np.random.seed(1)
idx = np.arange(len(y))
np.random.shuffle(idx)
X, y = X[idx,:], y[idx]  

X.shape, y.shape 

((70000, 784), (70000,))

In [ ]:
# check if a column is all 0
id = [np.sum(Xtrain[:,i] != 0)>10 for i in range(28**2)]
# number of columns that mostly zero
print(28**2-np.sum(id))
# number of columns that has at least more than 10 non-zero
np.sum(id)
X = X[:, id]

0


## Mforest

In [ ]:
# run 6 hours giving no result
G = 10
e20_mforest = compute_err_mforest(X, y, G, 0.2)
e20_mforest

In [ ]:
e35_mforest = compute_err_mforest(X, y, G, 0.35)
e50_mforest = compute_err_mforest(X, y, G, 0.5)
e65_mforest = compute_err_mforest(X, y, G, 0.65)
e80_mforest = compute_err_mforest(X, y, G, 0.8)

## DPER

In [ ]:
G = 10
e20_dper = compute_err_dper(X, y, G, 0.2)
e35_dper = compute_err_dper(X, y, G, 0.35)
e50_dper = compute_err_dper(X, y, G, 0.5)
e65_dper = compute_err_dper(X, y, G, 0.65)
e80_dper = compute_err_dper(X, y, G, 0.8)

computation time 1906.490648984909
computation time 1611.3365874290466
computation time 1387.5780763626099
computation time 1170.8552837371826
computation time 1039.692412853241


In [ ]:
# the left column is the error, the right column is the running time
np.vstack((e20_dper,e35_dper,e50_dper,e65_dper,e80_dper))

array([[2.92527404e-05, 1.90649065e+03],
       [3.89047417e-05, 1.61133659e+03],
       [4.91157708e-05, 1.38757808e+03],
       [5.82590926e-05, 1.17085528e+03],
       [7.32972844e-05, 1.03969241e+03]])

## Soft impute

In [ ]:
e20_soft = compute_err_soft(X, y, G, 0.2)
e35_soft = compute_err_soft(X, y, G, 0.35)
e50_soft = compute_err_soft(X, y, G, 0.5)
e65_soft = compute_err_soft(X, y, G, 0.65)
e80_soft = compute_err_soft(X, y, G, 0.8)

In [ ]:
# the left column is the error, the right column is the running time
np.vstack((e20_soft, e35_soft,e50_soft,e65_soft,e80_soft))
np.round(np.vstack((e20_soft, e35_soft,e50_soft,e65_soft,e80_soft))[0],3)

array([[3.89397495e-05, 1.02333118e+03],
       [6.64847559e-05, 1.31015335e+03],
       [9.57025225e-05, 1.74412689e+03],
       [1.30314425e-04, 2.28746755e+03],
       [1.69178211e-04, 2.65371597e+03]])

## MICE impute

In [ ]:
e20_mice = compute_err_mice(X, y, G, 0.2)

In [ ]:
e20_mice

In [ ]:
e35_mice = compute_err_mice(X, y, G, 0.35)

In [ ]:
e35_mice

In [ ]:
e50_mice = compute_err_mice(X, y, G, 0.5)

In [ ]:
e50_mice

In [ ]:
e65_mice = compute_err_mice(X, y, G, 0.65)

In [ ]:
e65_mice

In [ ]:
e80_mice = compute_err_mice(X, y, G, 0.8)

In [ ]:
e80_mice

## Nuclear 

In [ ]:
#kernel dies
e20_nuclear = compute_err_nuclear(X, y, G, 0.2)

In [ ]:
e20_nuclear

In [ ]:
e35_nuclear = compute_err_nuclear(X, y, G, 0.35)

In [ ]:
e35_nuclear

In [ ]:
e50_nuclear = compute_err_nuclear(X, y, G, 0.5)

In [ ]:
e50_nuclear

In [ ]:
e65_nuclear = compute_err_nuclear(X, y, G, 0.65)

In [ ]:
e65_nuclear

In [ ]:
e80_nuclear = compute_err_nuclear(X, y, G, 0.8)

In [ ]:
e80_nuclear